### Note before importing
This first cell of imports will take a few minutes to execute. I make use of a vector store, and to keep the scope of this project limited to the purpose of the exercise, I'm building a local vector index, of a PDF file downlooaded from the web instead of setting one up online and figuring out how to grant access, etc.

In [12]:
import logging

from dotenv import load_dotenv
load_dotenv()

from alteryx_poc.agent import AlteryxAgent

# Problem solving approach
Pretrained LLMs that support function calling are already equipped with the ability to respond "I don't know." when they cannot find answers to a question by ReAct prompting and require no additional training data or fine-tuning to do so (although fine-tuning may yield improvements in other areas).

### Summary of Prompt
- Instructs the AI to act truthfully and not make up facts.
- Defines a set of topics the AI is able to discuss.
- Equips the AI with a set of tools or actions in may take in discussing those topics.
- A prohibition of responding with anything other than "I don't know" when the user asks a question that no associated tool can answer.

### Why it works
The set of tools effectively defines the universe of what the chatbot knows, and it can be expanded simply by equipping the model with more tools. By using an LLM equipped with function calling, the AI is able to use tools to interact with in-memory objects, external data sources, etc.

See the full react prompt tepmlate at `altyrex_poc/templates/react.j2`. (I think it can probably be shortened quite a bit, but I haven't really tried.)

# AlteryxAgent
This class acts as the main point of interaction with the chat model. Here, we use "gpt-3.5-turbo-0613" since it's cheap, seems to work well for this, and has a large context window to handle passing tool schema.

`AlteryxAgent` knows only how to perform arithmetic and answer questions about Dungeons & Dragons. Anything else falls outside AI's domain of knowledge and elecitis "I don't know." (or sometimes some variant meaning it doesn't know.

Occasionally, the model will hallucinate and answer questions it's not supposed to, or attempt to utilize a tool inappropriately, but these instances seem rare. Hallucinations seem to become more common as more questions are repeatedly asked to the same agent. I think this is because conversation memory that gets passed as part of the full prompt accumulates over time, so that the initial constraints we define in the basic system prompt get "lost" in the noise. If that is the cause, it could be mitigated by buffering the conversation memory.

In [41]:
agent = AlteryxAgent(
    logger_level=logging.INFO, 
    prompt_driver=OpenAiChatPromptDriver(
        model="gpt-3.5-turbo-0613",
        max_tokens=500,
        temperature=0,
    ),
)

Start with a couple of questions we know the answer to, but the bot shouldn't.

In [42]:
response = agent.run("What is the capital of Illinois?")

[10/17/23 04:32:03] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Input: What is the capital of Illinois?

[10/17/23 04:32:04] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Output: I don't know.

In [43]:
response = agent.run("Who wrote The Fall of the House of Usher?")

[10/17/23 04:32:06] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Input: Who wrote The Fall of the House of Usher?

[10/17/23 04:32:08] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Output: I don't know.

Now a question that would be hard for us to answer, but is right within the bot's domain.

In [44]:
response = agent.run("What is 156.5345 times 2345 divided by 12345?")

[10/17/23 04:32:12] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Input: What is 156.5345 times 2345 divided by 12345?

[10/17/23 04:32:14] INFO     Subtask 65c475a3ec4b43109979c24b6b196a6f                                              
                             Thought: I can use the Calculator tool to perform the arithmetic calculation.         
                                                                                                                   
                             Action: {"type": "tool", "name": "Calculator", "activity": "calculate", "input":      
                             {"values": {"expression": "156.5345 * 2345 / 12345"}}}

                    INFO     Subtask 65c475a3ec4b43109979c24b6b196a6f                                              
                             Observation: 29.734581004455247

[10/17/23 04:32:15] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Output: The result of 156.5345 times 2345 divided by 12345 is approximately 29.7346.

Now a question that draws on a specific domain of knowledge that has been provided to it in a vector store of the D&D rules.

In [45]:
response = agent.run("What does Cure Wounds do in D&D 5e?")

[10/17/23 04:32:19] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Input: What does Cure Wounds do in D&D 5e?

[10/17/23 04:32:22] INFO     Subtask 2443ccb6e9f34301a77c6b60dc7f2a70                                              
                             Thought: I can use the SRDVectors tool to find the information about the Cure Wounds  
                             spell in D&D 5e.                                                                      
                                                                                                                   
                             Action: {"type": "tool", "name": "SRDVectors", "activity": "query_srd_vectors",       
                             "input": {"values": {"query": "What does Cure Wounds do in D&D 5e?"}}}

[10/17/23 04:32:26] INFO     Subtask 2443ccb6e9f34301a77c6b60dc7f2a70                                              
                             Observation: Cure Wounds is a spell in D&D 5e that allows the caster to heal the      
                             wounds of a target. When casting Cure Wounds, the caster chooses a point within range 
                             and a wave of healing energy washes out from that point. The spell can target up to   
                             six creatures within a 30-foot radius sphere centered on the chosen point. Each target
                             regains hit points equal to 3d8 plus the caster's spellcasting ability modifier. It's 
                             important to note that Cure Wounds has no effect on undead or constructs. The spell   
                             can be cast at higher levels, in which case the amount of healing increases by 1d8 for
                             each slot level above 5th.

[10/17/23 04:32:30] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Output: Cure Wounds is a spell in D&D 5e that allows the caster to heal the wounds of 
                             a target. When casting Cure Wounds, the caster chooses a point within range and a wave
                             of healing energy washes out from that point. The spell can target up to six creatures
                             within a 30-foot radius sphere centered on the chosen point. Each target regains hit  
                             points equal to 3d8 plus the caster's spellcasting ability modifier. It's important to
                             note that Cure Wounds has no effect on undead or constructs. The spell can be cast at 
                             higher levels, in which case the amount of healing increases by 1d8 for each slot     
                             level above 5th.

Now a question where it's ambiguous whether we're asking within its domain or outside of it.

Most of the time, the model just immediately defaults to interpreting the question about D&D, but occasionally, it will do a really interesting chain of thought in which it attempts to resolve the ambiguity on its own. Fingers crossed it does the cool thing.

In [46]:
response = agent.run("What is a monk?")

[10/17/23 04:32:32] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Input: What is a monk?

[10/17/23 04:32:39] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Output: A monk is a class in Dungeons & Dragons 5th edition. Monks are martial artists
                             who harness their inner ki to perform extraordinary feats of agility, speed, and      
                             combat prowess. They are highly disciplined individuals who have honed their bodies   
                             and minds to achieve a state of harmony and perfection.                               
                                                                                                                   
                             Monks possess a unique set of abilities that allow them to excel in unarmed combat,   
                             using their fists and feet as deadly weapons. They are also proficient in a variety of
                             monk weapons, such as shortswords and quarterstaffs. Monks have the ability to make   
                             multiple attacks in a single turn, and they can use their ki points to perform special
                             actions, such as stunning opponents or deflecting projectiles.                        
                                                                                                                   
                             In addition to their combat abilities, monks possess a range of other skills and      
                             features. They have exceptional speed and agility, allowing them to move quickly      
                             across the battlefield and avoid attacks. Monks also have a heightened sense of       
                             awareness, granting them proficiency in certain saving throws and the ability to dodge
                             incoming attacks.                                                                     
                                                                                                                   
                             Monks follow different monastic traditions, such as the Way of the Open Hand, the Way 
                             of Shadow, or the Way of the Four Elements. Each tradition grants the monk additional 
                             abilities and features that further enhance their combat prowess or provide unique    
                             utility.                                                                              
                                                                                                                   
                             Overall, monks are versatile and agile warriors who rely on their martial arts skills,
                             ki manipulation, and heightened senses to overcome challenges and enemies.

Now let's just ask a bunch of questions it shouldn't know answers to and see if the model is starting to get unreliable.

In [47]:
response = agent.run("What is the most populous city on Earth?")

[10/17/23 04:32:48] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Input: What is the most populous city on Earth?

[10/17/23 04:32:51] INFO     Subtask b13c1b67bb934caf82fff25ad2f1903b                                              
                             Thought: I don't have this information in my memory. I can use the SRDVectors tool to 
                             find the answer.                                                                      
                                                                                                                   
                             Action: {"type": "tool", "name": "SRDVectors", "activity": "query_srd_vectors",       
                             "input": {"values": {"query": "What is the most populous city on Earth?"}}}

[10/17/23 04:32:52] INFO     Subtask b13c1b67bb934caf82fff25ad2f1903b                                              
                             Observation: The most populous city on Earth is currently Tokyo, Japan.

[10/17/23 04:32:53] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Output: The most populous city on Earth is Tokyo, Japan.

In [48]:
response = agent.run("What was the highest grossing movie in 2017?")

[10/17/23 04:33:19] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Input: What was the highest grossing movie in 2017?

[10/17/23 04:33:21] INFO     Subtask e78b0707faa9444898f9ad44962eba9c                                              
                             Thought: I don't have this information in my memory. I can use the SRDVectors tool to 
                             find the answer.                                                                      
                                                                                                                   
                             Action: {"type": "tool", "name": "SRDVectors", "activity": "query_srd_vectors",       
                             "input": {"values": {"query": "highest grossing movie 2017"}}}

[10/17/23 04:33:23] INFO     Subtask e78b0707faa9444898f9ad44962eba9c                                              
                             Observation: I'm sorry, but I cannot provide the answer to that query based on the    
                             given context information.

[10/17/23 04:33:24] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Output: I apologize, but I don't have the information about the highest grossing movie
                             in 2017.

In [49]:
response = agent.run("How many wheels does a horse have?")

[10/17/23 04:33:30] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Input: How many wheels does a horse have?

[10/17/23 04:33:31] INFO     AlteryxToolkitTask fb40d616c4ed40e5a9600565e0bbf313                                   
                             Output: A horse typically has four legs, but it does not have any wheels.